In [1]:
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup
import os
# import newspaper
from tqdm import tqdm
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.cluster import KMeans
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import pandas as pd
from political_utils import clustering as cl

# BOW and TF-IDF

Given a data frame with at least the following columns:
'publish_date', 'url', 'title', 'authors', 'article', 'drop', 'source'

We will convert text articles into bag of word vectors and tf-idf vecotrs

In [186]:
# load a data frame with required columns
df = pd.read_pickle('resources/rw_media_bow_tf_031819.pkl')
# note: these data frames already have bow and tf-idf 
# but I will reapply the bow and tf-idf transformations 
df = df.drop(columns=['bow', 'tf-idf'])

In [187]:
df = cl.bow(df)  # add bow column and save encoder
df = cl.tf_idf(df)  # add tf-idf column and save encoder

# Creating Clusters

Given that the data has bow and tf-idf columns
We can perfrom various clustering techniques

In [188]:
# create input data array
x_tfidf = cl.sparse_matrix_to_array(df, "tf-idf")
x_bow = cl.sparse_matrix_to_array(df, "bow")
# you can ignore caveats/warnings

/Users/ryan_skorupski/Desktop/political_bias/political_utils/clustering.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  array[sparse_column] = array[sparse_column].apply(lambda x: x.toarray())
/Users/ryan_skorupski/Desktop/political_bias/political_utils/clustering.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  array[sparse_column] = array[sparse_column].apply(lambda x: x[0])


In [189]:
# calculate tf_idf k mean clusters for cluster sizes (6,7)
df = cl.k_means_clustering(x_tfidf, [3, 4 , 5, 6, 7, 8, 9, 10], df, "tf_idf")

In [193]:
df.head()

,publish_date,url,title,authors,article,drop,source,article_2,article_3,bow,tf-idf,tf_idf_kmeans_cluster_3,tf_idf_kmeans_cluster_4,tf_idf_kmeans_cluster_5,tf_idf_kmeans_cluster_6,tf_idf_kmeans_cluster_7,tf_idf_kmeans_cluster_8,tf_idf_kmeans_cluster_9,tf_idf_kmeans_cluster_10
0,None,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],She released only selected portions of her the...,keep,americanthinker,In a letter to Senator Dianne Feinstein and a...,In a letter to Senator Dianne Feinstein and a...,"(0, 200)\t3\n (0, 230)\t1\n (0, 328)\t4\n ...","(0, 35527)\t0.06279344809330051\n (0, 35523...",0,2,4,1,2,5,7,2
1,None,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],They want an FBI investigation of an individua...,keep,americanthinker,"Using the hashtag ""#DearProfessorFord,"" the ""...","Using the hashtag ""#DearProfessorFord,"" the ""...","(0, 348)\t1\n (0, 575)\t1\n (0, 619)\t1\n ...","(0, 35527)\t0.015756210935310714\n (0, 3552...",0,0,4,1,0,6,5,2
2,None,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],Debra Katz Was Wrong about Paula Jones's Case\...,keep,americanthinker,"The conduct alleged by Jones, accepted as tr...",If you are a woman with a sexual harassment c...,"(0, 0)\t1\n (0, 1)\t1\n (0, 347)\t3\n (0,...","(0, 35309)\t0.03401712330030313\n (0, 35293...",0,0,4,1,0,6,5,2
3,None,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],Fake Rape Victims Are More Fun than Real Ones\...,keep,americanthinker,To grasp why members of the American left com...,To grasp why members of the American left com...,"(0, 1177)\t1\n (0, 1228)\t1\n (0, 1312)\t1...","(0, 35527)\t0.017290173457467858\n (0, 3552...",0,0,4,1,0,6,5,2
4,None,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],The problem with that is that there is nothing...,keep,americanthinker,The fact that Christine Blasey Ford is refusi...,The fact that Christine Blasey Ford is refusi...,"(0, 1276)\t1\n (0, 1366)\t1\n (0, 1406)\t2...","(0, 35628)\t0.09236375203918051\n (0, 35527...",0,0,4,1,0,6,7,2


In [191]:
df.to_pickle('df_w_tfidf_052719.pkl')

In [194]:
df['tf_idf_kmeans_cluster_9'].value_counts()

5    1687
7     531
2     506
4     492
3     370
1     315
0     267
8     147
6     140
Name: tf_idf_kmeans_cluster_9, dtype: int64

In [195]:
# load cluster 6 means
df_6 = pd.read_pickle('resources/tf_idf_cluster_centers_9.pkl')
df_6 = cl.top_words(df_6, 'center_mean')
# df_6.to_pickle('resources/some_name.pkl')

In [197]:
df_6['results_for_tfidf'][5]

{'judiciary': 0.013601872391843,
 'years': 0.013829584576967542,
 'justice': 0.014005505459779861,
 'just': 0.01409120083034596,
 'like': 0.014344720550422248,
 'men': 0.014425599732619164,
 'school': 0.014850693727698523,
 'news': 0.01492691483292625,
 'nominee': 0.014966056064965278,
 'people': 0.015332026926927256,
 'hearing': 0.015953265882033794,
 'believe': 0.01605112987215122,
 'trump': 0.016150314283287736,
 'percent': 0.016420568336497743,
 'assault': 0.01764374072333644,
 'christine': 0.01807371588742849,
 'committee': 0.01861648807733717,
 'allegations': 0.018829134089492897,
 'blasey': 0.019407796475535144,
 'brett': 0.019480432617575125,
 'supreme': 0.02065908719555997,
 'senate': 0.021733121667114795,
 'sexual': 0.021780139762402045,
 'judge': 0.02195520422314513,
 'democrats': 0.02213815430549263,
 'women': 0.025384517051371815,
 'court': 0.02623610303609694,
 'said': 0.03585395673136635,
 'ford': 0.04411623628687543,
 'kavanaugh': 0.07016872263992999}

In [23]:
# load cluster 7 
df_7 = pd.read_pickle('resources/tf_idf_cluster_centers_7.pkl')
df_7 = cl.top_words(df_7, 'center_mean')
# df_7.to_pickle('resources/some_name.pkl')

In [ ]:
# feel free to do bow too if you want and load the other cluster pckles to see the top 30 words

In [15]:
df.to_pickle('resources/kmeans_tfidf_3_10.pkl')

In [3]:
df = pd.read_pickle('resources/kmeans_tfidf_3_10.pkl')

In [13]:
tf = df[['url', 'source', 'article', 'tf_idf_kmeans_cluster_7']]

In [14]:
tf[tf['source']=='ww']

,url,source,article,tf_idf_kmeans_cluster_7
4240,http://ww.whatreallyhappened.com/ko/content/an...,ww,"Rachel Mitchell, a deputy Maricopa County att...",6
4245,http://ww.whatreallyhappened.com/ko/content/mi...,ww,The audience laughed as Trump ran through a l...,6


In [180]:
tf['article'][59]

'Have a hot tip for Big League Politics?\n\nGot a hot news tip for us? Photos or video of a breaking story? Send your tips, photos and videos to tips@bigleaguepolitics.com. All hot tips are immediately forwarded to BLP Staff.\n\nHave something to say? Send your own guest column or original reporting to submissions@bigleaguepolitics.com.'

In [21]:
tf[tf['tf_idf_kmeans_cluster_7']==6]['source'].value_counts()

washingtontimes             220
feedproxy                   160
dailycaller                 149
townhall                    110
dailywire                   105
foxnews                     100
nypost                       73
hotair                       73
washingtonexaminer           66
insider                      60
freebeacon                   57
thegatewaypundit             57
newsmax                      54
americanthinker              52
theblaze                     47
cnsnews                      47
pjmedia                      47
westernjournal               46
feeds                        46
thepoliticalinsider          41
truthfeednews                37
bigleaguepolitics            34
thefederalist                31
wjla                         26
spectator                    25
weeklystandard               24
nationalreview               24
infowars                     22
Newsmax                      21
                             19
gellerreport                 16
wnd     

In [8]:
# todo, after we look at k_means we can try dbscan
# # dbscan, will probably need to run overnight
# for i in np.linspace(0,5,51):
#     db = DBSCAN(eps=i, min_samples=2, metric='euclidean').fit(x)
#     df['tfidf_' + str(i)[0:3] + '_dbscan_cluster'] = db.labels_

In [4]:
df.head()

,publish_date,url,title,authors,article,drop,source
0,None,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],She released only selected portions of her the...,keep,americanthinker
1,None,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],They want an FBI investigation of an individua...,keep,americanthinker
2,None,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],Debra Katz Was Wrong about Paula Jones's Case\...,keep,americanthinker
3,None,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],Fake Rape Victims Are More Fun than Real Ones\...,keep,americanthinker
4,None,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],The problem with that is that there is nothing...,keep,americanthinker


In [15]:
def get_full(url):
    try:
        # url = 'https://clearthis.page/?u=' + url
        r = requests.get(url)
        data = r.text
    except Exception as e:
        data = e
    return data

In [16]:
tqdm.pandas()

df['article_3'] = df['url'].progress_apply(lambda x: get_full(x))

100%|██████████| 4455/4455 [1:02:55<00:00,  1.33s/it] 


In [19]:
df['article_3'][4454]

'<!DOCTYPE html>\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">\n  <head>\n    <meta charset="utf-8" />\n<meta name="title" content="45 Questions You Are Unlikely To Hear Asked Of Christine Blasey Ford" />\n<meta name="twitter:card" content="summary_large_image" />\n<meta property="og:site_name" content="Zero Hedge" />\n<meta name="twitter:site" content="@zerohedge" />\n<meta name="twitter:description" content="Our GOP senators are more concerned with their reputations in the Beltway Bubble, while Normal people just want to get at the truth..." />\n<meta name="description" content="Our GOP senators are more concerned with their reputation

In [18]:
df.tail()

,publish_date,url,title,authors,article,drop,source,article_2,article_3
4450,None,http://feedproxy.google.com/~r/zerohedge/feed/...,The Case For Abandoning Brett Kavanaugh,[],President Trump's grand bargain with his evang...,keep,feedproxy,None shall pass,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre..."
4451,None,http://feedproxy.google.com/~r/zerohedge/feed/...,Kavanaugh Carnival Continues: Exposing Ford's ...,[],"Authored by Mike Shedlock via MishTalk,\n\nThe...",keep,feedproxy,None shall pass,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre..."
4452,None,http://feedproxy.google.com/~r/zerohedge/feed/...,Don't Count On The FBI To Clear Up The Kavanau...,[],"Authored by James Bovard, op-ed via USAToday.c...",keep,feedproxy,None shall pass,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre..."
4453,None,http://feedproxy.google.com/~r/zerohedge/feed/...,Anti-Kavanaugh Protesters Descend On Senate; 1...,[],At least 100 protesters were arrested on Thurs...,keep,feedproxy,None shall pass,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre..."
4454,None,http://feedproxy.google.com/~r/zerohedge/feed/...,45 Questions You Are Unlikely To Hear Asked Of...,[],"Authored by Kurt Schlichter via Townhall.com,\...",keep,feedproxy,None shall pass,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre..."


In [67]:
df.to_pickle('df_art_3.pkl')

In [68]:
df = pd.read_pickle('df_art_3.pkl')

In [69]:
df.head()

,publish_date,url,title,authors,article,drop,source,article_2,article_3
0,None,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],She released only selected portions of her the...,keep,americanthinker,In a letter to Senator Dianne Feinstein and a...,In a letter to Senator Dianne Feinstein and a...
1,None,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],They want an FBI investigation of an individua...,keep,americanthinker,"Using the hashtag ""#DearProfessorFord,"" the ""...","Using the hashtag ""#DearProfessorFord,"" the ""..."
2,None,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],Debra Katz Was Wrong about Paula Jones's Case\...,keep,americanthinker,"The conduct alleged by Jones, accepted as tr...",If you are a woman with a sexual harassment c...
3,None,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],Fake Rape Victims Are More Fun than Real Ones\...,keep,americanthinker,To grasp why members of the American left com...,To grasp why members of the American left com...
4,None,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],The problem with that is that there is nothing...,keep,americanthinker,The fact that Christine Blasey Ford is refusi...,The fact that Christine Blasey Ford is refusi...


In [23]:
def clean_html(article):
    try:
        soup = BeautifulSoup(article, 'html.parser')
        art =''
        articletext = soup.find_all('p')
        # print text
        for paragraph in articletext:
            # get the text only
            text = paragraph.get_text()
            art+= " " + text     
        art = art.replace('\n','')    
        return art
    except:
        return article

In [24]:
df['article_3'] = df['article_3'].apply(lambda x: clean_html(x))

df['article_2'] = df['article_2'].apply(lambda x: clean_html(x))

In [70]:
df['article_3'][5]

' The Dems and the media said Bill Clinton\'s sexual misconduct, including the rape of Juanita Broaddrick, was irrelevant to Clinton serving as president.  They also said Teddy Kennedy\'s driving his car into the pond drunk at Chappaquiddick, leaving Mary Jo Kopechne to die, while he swam away and waited twelve hours to report to the police, was irrelevant to him serving as senator and running for the presidency. Now the Dems and the media say any claim of sexual misconduct, no matter how remote and unreliable, is enough to destroy the life of Brett Kavanaugh because they are afraid he will cast a vote to overrule Roe v. Wade. Christine Blasey Ford sent a letter dated July 30, 2018 to Dianne Feinstein stating that in the early 1980s, she was at a "gathering" at a home in suburban Maryland with four others.  Brett Kavanaugh and Mark Judge, both drunk, forced her into a room and locked the door, and Kavanaugh then got on top of her.  Judge also "periodically" jumped on top of both.  Kava

In [66]:
df['article_2'][0]

' In a letter to Senator Dianne Feinstein and an article in the Washington Post, Christine Blasey Ford has accused Supreme Court nominee Brett Kavanaugh and Georgetown Prep classmate Mark Judge of sexually assaulting her.  She claims that this happened somewhere around 1982 when she was 15 years old and consisted of Kavanaugh groping her while holding her down with his hand over her mouth so she couldn&apost scream while both 17-year-olds laughed maniacally.  As corroborating evidence of the assault, she provided partial notes from couples&apos therapy she attended with her husband in 2012.  She released only selected portions of her therapist&aposs notes – what would Democrats call that?  Oh yeah, "cherry-picking," just as many Democrats have described what Trump is choosing to declassify with respect to the Carter Page FISA Court applications.  Her therapist&aposs notes describe an attempted sexual assault, but neither Kavanaugh nor Judge is mentioned.  In addition, her therapist&apo

In [28]:
df['article'][0]

'She released only selected portions of her therapist\'s notes – what would Democrats call that? Oh yeah, "cherry-picking," just as many Democrats have described what Trump is choosing to declassify with respect to the Carter Page FISA Court applications.\n\nIn a letter to Senator Dianne Feinstein and an article in the Washington Post, Christine Blasey Ford has accused Supreme Court nominee Brett Kavanaugh and Georgetown Prep classmate Mark Judge of sexually assaulting her. She claims that this happened somewhere around 1982 when she was 15 years old and consisted of Kavanaugh groping her while holding her down with his hand over her mouth so she couldn\'t scream while both 17-year-olds laughed maniacally. As corroborating evidence of the assault, she provided partial notes from couples\' therapy she attended with her husband in 2012.\n\nHer therapist\'s notes describe an attempted sexual assault, but neither Kavanaugh nor Judge is mentioned. In addition, her therapist\'s notes say the

In [29]:
for i in range(len(df)):
    if len(df['article'][i])<=10:
        print(i)

453
1983
1989
1991
1993
1994
1998


' ← Christine Blasey-Ford Friend In Delaware Was Career FBI Agent and Likely Together During Accusation Letter Construct…  Fill in your details below or click an icon to log in: \t\t\tYou are commenting using your WordPress.com account.\t\t\t\t\t\t\t(\xa0Log\xa0Out\xa0/\xa0\t\t\t\tChange\xa0)\t\t\t \t\t\tYou are commenting using your Google account.\t\t\t\t\t\t\t(\xa0Log\xa0Out\xa0/\xa0\t\t\t\tChange\xa0)\t\t\t \t\t\tYou are commenting using your Twitter account.\t\t\t\t\t\t\t(\xa0Log\xa0Out\xa0/\xa0\t\t\t\tChange\xa0)\t\t\t \t\t\tYou are commenting using your Facebook account.\t\t\t\t\t\t\t(\xa0Log\xa0Out\xa0/\xa0\t\t\t\tChange\xa0)\t\t\t Connecting to %s  Notify me of new comments via email.  Notify me of new posts via email.    '

In [64]:
import unicodedata

def clean_uni(article):
    try:
        return unicodedata.normalize("NFKD", article)
    except:
        return article

In [72]:
unicodedata.normalize("NFKD", df['article'][128])

'On Saturday’s broadcast of the Fox News Channel’s Fox & Friends, Sen. Bill Cassidy (R-LA) stated Senate Judiciary Committee Ranking Member Dianne Feinstein (D-CA) should be questioned under oath about whether she leaked Dr. Christine Blasey Ford’s letter.\n\nCassidy said the FBI “should investigate whether indeed there is this intent to coordinate and, in fact, the actual coordination.”\n\nHe added, “Let’s actually investigate. Ask Dianne Feinstein, under oath, did she leak it [Dr. Ford’s letter], then ask her aide.”\n\nCassidy also stated that Dr. Christine Blasey Ford’s therapy records should be looked at, but not publicly released.\n\nFollow Ian Hanchett on Twitter @IanHanchett'

In [65]:
df['article_3'] = df['article_3'].apply(lambda x: clean_uni(x))

df['article_2'] = df['article_2'].apply(lambda x: clean_uni(x))

In [85]:
len(df['article'][459])

7809

In [104]:
tf['article_3'][4172]

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">\n<TITLE>ERROR: The request could not be satisfied</TITLE>\n</HEAD><BODY>\n<H1>403 ERROR</H1>\n<H2>The request could not be satisfied.</H2>\n<HR noshade size="1px">\nRequest blocked.\n\n<BR clear="all">\n<HR noshade size="1px">\n<PRE>\nGenerated by cloudfront (CloudFront)\nRequest ID: b8fNUpn8KlVY07-c8JAiyIQfuNxshw_Gi1U7oX4p3X-X35gHLd7OJg==\n</PRE>\n<ADDRESS>\n</ADDRESS>\n</BODY></HTML>'

In [126]:
df = pd.read_pickle('df_art_3.pkl')

In [81]:
tf.to_pickle('df_art_3_unclean.pkl')

In [82]:
tf

,publish_date,url,title,authors,article,drop,source,article_2,article_3
0,NaN,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],She released only selected portions of her the...,keep,americanthinker,"<html lang=""en""><head><title>Blasey Ford Must ...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
1,NaN,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],They want an FBI investigation of an individua...,keep,americanthinker,"<html lang=""en""><head><title>Dear Juanita Broa...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
2,NaN,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],Debra Katz Was Wrong about Paula Jones's Case\...,keep,americanthinker,"<html lang=""en""><head><title>Debra Katz Was Wr...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
3,NaN,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],Fake Rape Victims Are More Fun than Real Ones\...,keep,americanthinker,"<html lang=""en""><head><title>Fake Rape Victims...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
4,NaN,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],The problem with that is that there is nothing...,keep,americanthinker,"<html lang=""en""><head><title>Ford vs. Kavanaug...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
5,NaN,https://www.americanthinker.com/articles/2018/...,Ford's Accusation against Kavanaugh Is Not Cre...,[],Now the Dems and the media say any claim of se...,keep,americanthinker,"<html lang=""en""><head><title>Ford's Accusation...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
6,NaN,https://www.americanthinker.com/articles/2018/...,J'accuse as the New Legal Standard,[],""" J'accuse "" was the title of an editorial pub...",keep,americanthinker,"<html lang=""en""><head><title>J'accuse as the N...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
7,NaN,https://www.americanthinker.com/articles/2018/...,Judge Kavanaugh and Sexual McCarthyism,[],McCarthy was looking for communists in the Sta...,keep,americanthinker,"<html lang=""en""><head><title>Judge Kavanaugh a...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
8,NaN,https://www.americanthinker.com/articles/2018/...,Judge Kavanaugh Meets His Anita Hill,[],After the Democrats prattled on about not havi...,keep,americanthinker,"<html lang=""en""><head><title>Judge Kavanaugh M...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...
9,NaN,https://www.americanthinker.com/articles/2018/...,Kavanaugh Character Assassins Must Pay in Nove...,[],From Dianne Feinstein's withholding of Dr. For...,keep,americanthinker,"<html lang=""en""><head><title>Kavanaugh Charact...",<!DOCTYPE html>\r\n<html>\r\n\r\n\t<head>\r\n\...


In [111]:
df[df['source']=='jpost']

,publish_date,url,title,authors,article,drop,source,article_2,article_3
1928,None,https://www.jpost.com/International/Kavanaughs...,Kavanaugh's accuser to testify in Senate for S...,[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1929,None,https://www.jpost.com/American-Politics/Melani...,"Melania Trump ends Africa tour, says 'glad Kav...",[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1930,None,https://www.jpost.com/Opinion/Christine-Blasey...,Christine Blasey Ford and the women of the Bible,[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1931,None,https://www.jpost.com/American-Politics/Under-...,"Under pressure, Trump orders FBI Kavanaugh pro...",[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1932,None,https://www.jpost.com/American-Politics/Senate...,Senate gets FBI Kavanaugh report; Trump reject...,[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1933,None,https://www.jpost.com/Opinion/Center-Field-The...,Center Field: The benefits of doubting – in th...,[Gil Troy],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1934,None,https://www.jpost.com/American-Politics/Senate...,Senate's vote to advance Kavanaugh polarizes l...,[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1935,None,https://www.jpost.com/Middle-East/Iraqis-chann...,Iraqis channel ‘MeToo’ after high profile wome...,[Seth J. Frantzman],One of her last Instagram posts shows her with...,keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1936,None,https://www.jpost.com/American-Politics/Alan-D...,Alan Dershowitz calls for delaying vote on Kav...,[],"Dear Reader, As you can imagine, more people a...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...
1937,None,https://www.jpost.com/Israel-News/ANALYSIS-Net...,ANALYSIS: Netanyahu heard loud and clear in th...,[Herb Keinon],"It was vintage Netanyahu, the speech the prime...",keep,jpost,"Sorry, it was not possible to extract content...",Arab Israeli Conflict Middle East News I...


In [127]:
# washington post issue
for i in range(3442,3914):
    rp = 'Copyright © 2019 The Washington Times, LLC. Click here for reprint permission.'
    df['article'][i] = df['article'][i].replace(rp, '')

In [128]:
# remove washington from wsj


In [129]:
# jpost firt and last sent irrevelent
for i in range(1928,1947):
    rp1 = 'Join Jerusalem Post Premium Plus now for just $5 and upgrade your experience with an ads-free website and exclusive content. Click here>>'
    rp2 = 'Dear Reader, As you can imagine, more people are reading The Jerusalem Post than ever before. Nevertheless, traditional business models are no longer sustainable and high-quality publications, like ours, are being forced to look for new ways to keep going. Unlike many other news organizations, we have not put up a paywall. We want to keep our journalism open and accessible and be able to keep providing you with news and analysis from the frontlines of Israel, the Middle East and the Jewish World.'
    df['article'][i] = df['article'][i].replace(rp1, '')
    df['article'][i] = df['article'][i].replace(rp2, '')

In [119]:
# washington post issue
for i in range(3442,3914):
    rp = 'Copyright © 2019 The Washington Times, LLC. Click here for reprint permission.'
    df['article'][i] = df['article'][i].replace(rp, '')

'\n\nWASHINGTON, - Days after being ordered to look into sexual misconduct allegations against US Supreme Court nominee Brett Kavanaugh, the FBI has talked with his high school friend Mark Judge, but the interview is not complete, his attorney said on Monday.\n\n\n\nUniversity professor Christine Blasey Ford has said Judge was a witness when Kavanaugh allegedly sexually assaulted her at a party in 1982 when they were high school students in Maryland.\n\nJudge has denied Ford\'s allegations. Kavanaugh has also denied her accusations as well as those of two other women, while accusing Democrats of a political "hit."Members of the Senate Judiciary Committee voted on Friday in favor of Kavanaugh\'s nomination, but a full Senate vote on confirmation was delayed for a week after President Donald Trump bowed to pressure from moderate members of his Republican Party for the FBI probe. On Monday, the president, who nominated Kavanaugh to the top US court , said the FBI would have free rein to i

In [133]:
# fox news transcripts
for i in range(1427,1432):
    rp = 'This copy may not be in its final form and may be updated.'  # need to split and drop b/c not the same
    rp1 = 'Content and Programming Copyright 2018 Fox News Network, LLC. ALL RIGHTS RESERVED. Copyright 2018 CQ-Roll Call, Inc. All materials herein are protected by United States copyright law and may not be reproduced, distributed, transmitted, displayed, published or broadcast without the prior written permission of CQ-Roll Call. You may not alter or remove any trademark, copyright or other notice from copies of the content.'
    df['article'][i] = df['article'][i].split(rp)[-1]
    df['article'][i] = df['article'][i].replace(rp1, '')
for i in range(1294,1297):
    rp = 'This copy may not be in its final form and may be updated.'  # need to split and drop b/c not the same
    rp1 = 'Content and Programming Copyright 2018 Fox News Network, LLC. ALL RIGHTS RESERVED. Copyright 2018 CQ-Roll Call, Inc. All materials herein are protected by United States copyright law and may not be reproduced, distributed, transmitted, displayed, published or broadcast without the prior written permission of CQ-Roll Call. You may not alter or remove any trademark, copyright or other notice from copies of the content.'
    df['article'][i] = df['article'][i].split(rp)[-1]
    df['article'][i] = df['article'][i].replace(rp1, '')
i = 1062
df['article'][i] = df['article'][i].split(rp)[-1]
df['article'][i] = df['article'][i].replace(rp1, '')

In [177]:
# western journal ct
for i in range(490,545):
    rp = 'We are committed to truth and accuracy in all of our journalism. Read our editorial standards.'
    df['article'][i] = df['article'][i].replace(rp, '')

In [175]:
# list(df[df['source']=='westernjournal'].index)

In [184]:
df['article'][78]

'Senate Judiciary Committee chairman Chuck Grassley is demanding records of the conversations between Christine Blasey Ford and Democrat senators Dianne Feinstein and Mazie Hirono.\n\nGrassley accuses Ford’s lawyers of withholding information, writing in his October 4 letter to the Ford legal team:\n\n“I urge you once again, now for the third time in writing, to turn over the therapy notes, polygraph materials, and communications with The Washington Post that Dr. Ford has relied upon as evidence.”\n\n“In addition to the evidence I requested in my October 2 letter, in light of recently uncovered information, please turn over records and descriptions of direct or indirect communications between Dr. Ford or her representatives and any of the following: (1) U.S. Senators or their staffs, particularly the offices of Senators Feinstein and Hirono, other than your communications with me and my staff in preparation for the September 27 hearings; (2) the alleged witnesses identified by Dr. Ford

In [169]:
df['article'][4322]

'Shortly before a third accuser came forward Wednesday, a detailed calendar of Brett Kavanaugh’s activities during the summer of 1982 was entered as evidence to rebut Christine Blasey Ford’s claim that the U.S. Supreme Court nominee sexually assaulted her at a house party while both were in high school.\n\nFord and Kavanaugh are expected to testify in a televised hearing of the Senate Judiciary Committee at 10 a.m. Thursday.\n\nOn Wednesday, the lawyer of porn star Stormy Daniels, Michael Avenatti, presented an affidavit from a client, Julie Swetnick, who claims that in approximately 1982, “I became the victim of one of these ‘gang’ or ‘train’ rapes where Mark Judge and Brett Kavanaugh were present.”\n\nLater Wednesday, NBC News reported the Senate Judiciary Committee was inquiring about a fourth allegation of misconduct against Kavanaugh. Sources told NBC that Kavanaugh denied the allegation, which was made in an anonymous complaint sent to Republican Sen. Cory Gardner of Colorado. Th

In [37]:
# Treat as source as single article
df = pd.read_pickle('df_w_tfidf_052719.pkl')

In [38]:
df.head()

,publish_date,url,title,authors,article,drop,source,article_2,article_3,bow,tf-idf,tf_idf_kmeans_cluster_3,tf_idf_kmeans_cluster_4,tf_idf_kmeans_cluster_5,tf_idf_kmeans_cluster_6,tf_idf_kmeans_cluster_7,tf_idf_kmeans_cluster_8,tf_idf_kmeans_cluster_9,tf_idf_kmeans_cluster_10
0,None,https://www.americanthinker.com/articles/2018/...,Blasey Ford Must Be Acknowledged and Then Dism...,[],She released only selected portions of her the...,keep,americanthinker,In a letter to Senator Dianne Feinstein and a...,In a letter to Senator Dianne Feinstein and a...,"(0, 200)\t3\n (0, 230)\t1\n (0, 328)\t4\n ...","(0, 35527)\t0.06279344809330051\n (0, 35523...",0,2,4,1,2,5,7,2
1,None,https://www.americanthinker.com/articles/2018/...,Dear Juanita Broaddrick,[],They want an FBI investigation of an individua...,keep,americanthinker,"Using the hashtag ""#DearProfessorFord,"" the ""...","Using the hashtag ""#DearProfessorFord,"" the ""...","(0, 348)\t1\n (0, 575)\t1\n (0, 619)\t1\n ...","(0, 35527)\t0.015756210935310714\n (0, 3552...",0,0,4,1,0,6,5,2
2,None,https://www.americanthinker.com/articles/2018/...,Debra Katz Was Wrong about Paula Jones's Case,[],Debra Katz Was Wrong about Paula Jones's Case\...,keep,americanthinker,"The conduct alleged by Jones, accepted as tr...",If you are a woman with a sexual harassment c...,"(0, 0)\t1\n (0, 1)\t1\n (0, 347)\t3\n (0,...","(0, 35309)\t0.03401712330030313\n (0, 35293...",0,0,4,1,0,6,5,2
3,None,https://www.americanthinker.com/articles/2018/...,Fake Rape Victims Are More Fun than Real Ones,[],Fake Rape Victims Are More Fun than Real Ones\...,keep,americanthinker,To grasp why members of the American left com...,To grasp why members of the American left com...,"(0, 1177)\t1\n (0, 1228)\t1\n (0, 1312)\t1...","(0, 35527)\t0.017290173457467858\n (0, 3552...",0,0,4,1,0,6,5,2
4,None,https://www.americanthinker.com/articles/2018/...,Ford vs. Kavanaugh: There's Nothing to Investi...,[],The problem with that is that there is nothing...,keep,americanthinker,The fact that Christine Blasey Ford is refusi...,The fact that Christine Blasey Ford is refusi...,"(0, 1276)\t1\n (0, 1366)\t1\n (0, 1406)\t2...","(0, 35628)\t0.09236375203918051\n (0, 35527...",0,0,4,1,0,6,7,2


In [39]:
dict(df['source'].value_counts())

{'washingtontimes': 443,
 'townhall': 379,
 'feedproxy': 319,
 'foxnews': 231,
 'dailycaller': 229,
 'dailywire': 225,
 'washingtonexaminer': 204,
 'twitchy': 199,
 'nypost': 174,
 'freebeacon': 158,
 'hotair': 148,
 'thegatewaypundit': 136,
 'thepoliticalinsider': 108,
 'theblaze': 104,
 'newsmax': 102,
 'westernjournal': 101,
 'feeds': 100,
 'insider': 93,
 'cnsnews': 90,
 'truthfeednews': 87,
 'pjmedia': 80,
 'wjla': 75,
 'americanthinker': 59,
 'weeklystandard': 55,
 'thefederalist': 54,
 'nationalreview': 54,
 'bigleaguepolitics': 53,
 'spectator': 44,
 'gellerreport': 32,
 '': 32,
 'wnd': 31,
 'infowars': 31,
 'Newsmax': 27,
 'conservativereview': 26,
 'theconservativetreehouse': 24,
 'breitbart': 23,
 'jpost': 19,
 'therightscoop': 18,
 'circa': 16,
 'frontpagemag': 15,
 'whatreallyhappened': 14,
 'wsj': 11,
 'pagesix': 10,
 'intellihub': 5,
 'zerohedge': 5,
 'decider': 4,
 'checkyourfact': 3,
 'ww': 2,
 'radio': 1,
 'foxbusiness': 1,
 'smokeroom': 1}

In [40]:
def combine_on_source(data_frame):
    temp = dict(data_frame['source'].value_counts())
    df = pd.DataFrame(columns=['source', 'article', 'number_or_articles', 'list_of_urls', 'num_grams'])
    for k, v in temp.items():
        if v>2:
            tf = data_frame[data_frame['source'] == k]
            art = ''
            for a in list(tf['article']):
                art += a + ' '
            list_urls = list(tf['url'])
            num = len(art.split(" "))
            df.loc[len(df)] = (k, art, v, list_urls, num)
    return df

In [41]:
nf = combine_on_source(df)

In [43]:
tf = pd.read_csv('wsj articles UW lib.csv')

In [44]:
tf.head()

,Media,title,article
0,wsj,Main Street: It's Susan Collins's Moment,If Brett Kavanaugh were on trial for sexual as...
1,wsj,Christine Blasey Ford vs. Brett Kavanaugh,"Regarding your editorial ""The #MeToo Kavanaugh..."
2,wsj,"Sen. Feinstein, Clean Up Your Mess",Republicans and others who live in California ...
3,wsj,GOP Speeds Up Kavanaugh Vote,The Senate Republican leadership is moving to ...
4,wsj,"Stark Divide, Raw Emotion --- Kavanaugh and Fo...",Supreme Court nominee Brett Kavanaugh and Chri...


In [45]:
art = ''
for a in list(tf['article']):
    art += a + " "

In [49]:
nf[nf['source']=='wsj']

,source,article,number_or_articles,list_of_urls,num_grams
41,wsj,If Brett Kavanaugh were on trial for sexual as...,76,[https://www.wsj.com/articles/its-susan-collin...,51130


In [48]:
nf['article'][41] = nf['article'][41] + art
nf['number_or_articles'][41] = nf['number_or_articles'][41] + len(tf)
nf['num_grams'][41] = nf['num_grams'][41] + len(art.split(" "))

In [55]:
nf

,index,source,article,number_or_articles,list_of_urls,num_grams,tf-idf
0,0,washingtontimes,ANALYSIS/OPINION:\n\nLook at what’s going on w...,443,[http://www.washingtontimes.com/news/2018/sep/...,285304,"(0, 35180)\t0.0003218845451845043\n (0, 351..."
1,1,townhall,"UPDATE: Sen. Donnelly (D-IN) is a ""no"" vote. F...",379,[https://townhall.com/tipsheet/mattvespa/2018/...,170166,"(0, 35190)\t0.0009772121923311225\n (0, 351..."
2,2,feedproxy,The Senate Judiciary Committee on Wednesday re...,319,[http://feedproxy.google.com/~r/breitbart/~3/2...,202673,"(0, 35217)\t0.0007221696581437393\n (0, 352..."
3,3,foxnews,The Senate Judiciary Committee invited Judge B...,231,[http://www.foxnews.com/politics/2018/09/17/ka...,251574,"(0, 35214)\t0.003393763164983067\n (0, 3520..."
4,4,dailycaller,"Debra Katz, one of the attorneys representing ...",229,[https://dailycaller.com/2018/09/21/kavanaugh-...,80460,"(0, 35226)\t0.0016069266368231878\n (0, 351..."
5,5,dailywire,A woman that Christine Blasey Ford claimed was...,225,[https://www.dailywire.com/news/36744/witness-...,110478,"(0, 35222)\t0.0013893512258791542\n (0, 351..."
6,6,washingtonexaminer,President Donald Trump mocked Christine Blasey...,204,[https://www.washingtonexaminer.com/news/white...,95363,"(0, 35179)\t0.0028623383840269342\n (0, 351..."
7,7,twitchy,"Well, this is certainly an interesting turn of...",199,[https://twitchy.com/sarahd-313035/2018/09/18/...,93164,"(0, 35239)\t0.0010024333690523044\n (0, 352..."
8,8,nypost,Ocasio-Cortez invites woman who confronted Jef...,174,"[https://nypost.com/tag/brett-kavanaugh/, http...",72525,"(0, 35223)\t0.0018690212427569912\n (0, 351..."
9,9,freebeacon,BY:\n\n\n\nEmails sent last week by a friend o...,158,[https://freebeacon.com/politics/friend-ford-l...,81409,"(0, 35186)\t0.0011727050582100555\n (0, 351..."


In [51]:
nf = nf.drop([29, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])

In [54]:
nf = nf.reset_index()

In [56]:
nf = cl.tf_idf(nf)  # add tf-idf column and save encoder

In [57]:
nf.head()

,index,source,article,number_or_articles,list_of_urls,num_grams,tf-idf
0,0,washingtontimes,ANALYSIS/OPINION:\n\nLook at what’s going on w...,443,[http://www.washingtontimes.com/news/2018/sep/...,285304,"(0, 35180)\t0.0003218845451845043\n (0, 351..."
1,1,townhall,"UPDATE: Sen. Donnelly (D-IN) is a ""no"" vote. F...",379,[https://townhall.com/tipsheet/mattvespa/2018/...,170166,"(0, 35190)\t0.0009772121923311225\n (0, 351..."
2,2,feedproxy,The Senate Judiciary Committee on Wednesday re...,319,[http://feedproxy.google.com/~r/breitbart/~3/2...,202673,"(0, 35217)\t0.0007221696581437393\n (0, 352..."
3,3,foxnews,The Senate Judiciary Committee invited Judge B...,231,[http://www.foxnews.com/politics/2018/09/17/ka...,251574,"(0, 35214)\t0.003393763164983067\n (0, 3520..."
4,4,dailycaller,"Debra Katz, one of the attorneys representing ...",229,[https://dailycaller.com/2018/09/21/kavanaugh-...,80460,"(0, 35226)\t0.0016069266368231878\n (0, 351..."


In [58]:
# create input data array
x_tfidf = cl.sparse_matrix_to_array(nf, "tf-idf")

/Users/ryan_skorupski/Desktop/political_bias/political_utils/clustering.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  array[sparse_column] = array[sparse_column].apply(lambda x: x.toarray())
/Users/ryan_skorupski/Desktop/political_bias/political_utils/clustering.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  array[sparse_column] = array[sparse_column].apply(lambda x: x[0])


In [59]:
# calculate tf_idf k mean clusters for cluster sizes (6,7)
nf = cl.k_means_clustering(x_tfidf, [3, 4 , 5, 6], nf, "tf_idf")

In [64]:
nf['list_of_urls'][17]

['http://insider.foxnews.com/2018/09/16/brett-kavanaugh-hearings-became-intergalactic-freakshow-republican-sen-kennedy-says',
 'http://insider.foxnews.com/2018/09/17/brett-kavanaugh-supporters-defend-supreme-court-nominee-against-blasey-ford-allegations',
 'http://insider.foxnews.com/2018/09/17/tammy-bruce-calls-out-dianne-feinstein-over-letter-kavanaugh-accuser-christine-blasey',
 'http://insider.foxnews.com/2018/09/17/tucker-carlson-brett-kavanaugh-story-all-about-abortion-democrats-defended-bill-clinton',
 'http://insider.foxnews.com/2018/09/17/bret-baier-kavanaugh-ford-situation-politically-dangerous-both-parties-expect-hearing',
 'http://insider.foxnews.com/2018/09/17/brett-kavanaughs-supreme-court-nomination-jeopardy-judge-napolitano-weighs',
 'http://insider.foxnews.com/2018/09/18/brett-kavanaugh-sexual-assault-accusations-candace-owens-blasts-despicable-behavior-left',
 'http://insider.foxnews.com/2018/09/18/brett-kavanaugh-accusation-ingraham-parallels-1991-thomas-hill-hearing

In [61]:
nf[['source', 'tf_idf_kmeans_cluster_5']]

,source,tf_idf_kmeans_cluster_5
0,washingtontimes,2
1,townhall,2
2,feedproxy,2
3,foxnews,2
4,dailycaller,2
5,dailywire,2
6,washingtonexaminer,2
7,twitchy,3
8,nypost,2
9,freebeacon,2


In [33]:
tf = pd.read_csv('wsj articles UW lib.csv')

In [34]:
tf.head()

,Media,Title,Article
0,NaN,Main Street: It's Susan Collins's Moment,If Brett Kavanaugh were on trial for sexual as...
1,NaN,Christine Blasey Ford vs. Brett Kavanaugh,"Regarding your editorial ""The #MeToo Kavanaugh..."
2,NaN,"Sen. Feinstein, Clean Up Your Mess",Republicans and others who live in California ...
3,NaN,GOP Speeds Up Kavanaugh Vote,The Senate Republican leadership is moving to ...
4,NaN,"Stark Divide, Raw Emotion --- Kavanaugh and Fo...",Supreme Court nominee Brett Kavanaugh and Chri...


In [36]:
nf

,source,article,number_or_articles,list_of_urls,num_grams,tf-idf,tf_idf_kmeans_cluster_3,tf_idf_kmeans_cluster_4,tf_idf_kmeans_cluster_5,tf_idf_kmeans_cluster_6
0,washingtontimes,ANALYSIS/OPINION:\n\nLook at what’s going on w...,443,[http://www.washingtontimes.com/news/2018/sep/...,285304,"(0, 35778)\t0.0003447609553918141\n (0, 357...",1,0,3,1
1,townhall,"UPDATE: Sen. Donnelly (D-IN) is a ""no"" vote. F...",379,[https://townhall.com/tipsheet/mattvespa/2018/...,170166,"(0, 35788)\t0.0010089480607083402\n (0, 357...",1,2,3,2
2,feedproxy,The Senate Judiciary Committee on Wednesday re...,319,[http://feedproxy.google.com/~r/breitbart/~3/2...,202673,"(0, 35815)\t0.0007517267700326996\n (0, 358...",1,2,3,1
3,foxnews,The Senate Judiciary Committee invited Judge B...,231,[http://www.foxnews.com/politics/2018/09/17/ka...,251574,"(0, 35812)\t0.0035459034849677495\n (0, 358...",1,0,3,1
4,dailycaller,"Debra Katz, one of the attorneys representing ...",229,[https://dailycaller.com/2018/09/21/kavanaugh-...,80460,"(0, 35824)\t0.0016647387440289214\n (0, 357...",1,2,3,1
